In [1]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import random
import math

# Window dimensions
window_width = 800
window_height = 600

# Game variables
ball_x = window_width // 2
ball_y = window_height // 2
ball_radius = 20
ball_speed = 5
health = 100
score = 0
snakes = []  # List of snakes
projectiles = []  # Projectiles shot by the ball
upgrade = False  # Ball upgrade status
game_over = False
is_paused = False
game_started = False ##

curvy_threshold = 5  # Snakes start moving curvy after reaching this score


# Drawing a pixel
def draw_pixel(x, y):
    glBegin(GL_POINTS)
    glVertex2f(x, y)
    glEnd()

# Midpoint Circle Algorithm
def draw_circle(xc, yc, r):
    x, y = 0, r
    d = 1 - r
    circle_points(xc, yc, x, y)
    while x < y:
        if d < 0:
            d += 2 * x + 3
        else:
            d += 2 * (x - y) + 5
            y -= 1
        x += 1
        circle_points(xc, yc, x, y)

def circle_points(xc, yc, x, y):
    points = [
        (xc + x, yc + y), (xc - x, yc + y),
        (xc + x, yc - y), (xc - x, yc - y),
        (xc + y, yc + x), (xc - y, yc + x),
        (xc + y, yc - x), (xc - y, yc - x)
    ]
    for point in points:
        draw_pixel(*point)


def draw_text_points(string, x, y): ##
    glColor3f(0, 0, 0)
    offset_x = x
    for char in string:
        char_data = glutStrokeCharacter(GLUT_STROKE_ROMAN, ord(char))
        glPushMatrix()
        glTranslatef(offset_x, y, 0)
        glScalef(0.0001, 0.0001, 1)
        for stroke in char_data:
            glBegin(GL_POINTS)
            for point in stroke:
                glVertex2f(point[0], point[1])
            glEnd()
        glPopMatrix()
        offset_x += 0.08 ##

def starting_interface():##
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()

    # Draw Start button using points
    glColor3f(0, 1, 0)
    glBegin(GL_POINTS)
    for x in range(-40, 41):
        for y in range(20, 41):
            glVertex2f(x / 100.0, y / 100.0)
    glEnd()

    # Draw Exit button using points
    glColor3f(1, 0, 0)
    glBegin(GL_POINTS)
    for x in range(-40, 41):
        for y in range(-40, -19):
            glVertex2f(x / 100.0, y / 100.0)
    glEnd()

    # Draw text on buttons
    draw_text_points("START", -0.2, 0.25)
    draw_text_points("EXIT", -0.15, -0.35)

    glutSwapBuffers()##





# Drawing a snake
def draw_snake(snake):
    # Define fixed dimensions for the snake
    body_length = 80  # Fixed length for the rectangle
    body_thickness = 20  # Fixed thickness for the rectangle

    # Determine the orientation based on movement direction
    if snake["dx"] != 0:  # Horizontal movement
        x1 = snake["x"] - body_length // 2
        x2 = snake["x"] + body_length // 2
        y1 = snake["y"] - body_thickness // 2
        y2 = snake["y"] + body_thickness // 2
    else:  # Vertical movement
        x1 = snake["x"] - body_thickness // 2
        x2 = snake["x"] + body_thickness // 2
        y1 = snake["y"] - body_length // 2
        y2 = snake["y"] + body_length // 2

    # Draw the rectangle body
    glBegin(GL_QUADS)
    glVertex2f(x1, y1)
    glVertex2f(x2, y1)
    glVertex2f(x2, y2)
    glVertex2f(x1, y2)
    glEnd()

    # Triangle head (snake's direction)
    if snake["dx"] > 0:  # Moving right
        triangle = [
            (x2, (y1 + y2) / 2),  # Left corner of the triangle base
            (x2 + body_thickness, y1),  # Bottom right of the base
            (x2 + body_thickness, y2)  # Top right of the base
        ]
    elif snake["dx"] < 0:  # Moving left
        triangle = [
            (x1, (y1 + y2) / 2),  # Right corner of the triangle base
            (x1 - body_thickness, y1),  # Bottom left of the base
            (x1 - body_thickness, y2)  # Top left of the base
        ]
    elif snake["dy"] > 0:  # Moving up
        triangle = [
            ((x1 + x2) / 2, y2),  # Bottom center of the triangle base
            (x1, y2 + body_thickness),  # Left corner of the triangle base
            (x2, y2 + body_thickness)  # Right corner of the triangle base
        ]
    else:  # Moving down
        triangle = [
            ((x1 + x2) / 2, y1),  # Top center of the triangle base
            (x1, y1 - body_thickness),  # Left corner of the triangle base
            (x2, y1 - body_thickness)  # Right corner of the triangle base
        ]

    # Draw the triangle head
    glBegin(GL_TRIANGLES)
    for vertex in triangle:
        glVertex2f(*vertex)
    glEnd()



# Spawn snakes
def spawn_snake(value=0):
    if not is_paused and not game_over:
        side = random.choice(['top', 'bottom', 'left', 'right'])
        size = random.randint(10, 30)
        speed = random.randint(2, 5)
        oscillation_speed = random.uniform(0.05, 0.2)  # Speed of the sine wave (only used after threshold)
        if side == 'top':
            x = random.randint(0, window_width)
            y = window_height
            dx, dy = 0, -speed
        elif side == 'bottom':
            x = random.randint(0, window_width)
            y = 0
            dx, dy = 0, speed
        elif side == 'left':
            x = 0
            y = random.randint(0, window_height)
            dx, dy = speed, 0
        elif side == 'right':
            x = window_width
            y = random.randint(0, window_height)
            dx, dy = -speed, 0
        snakes.append({
            "x": x, 
            "y": y, 
            "size": size, 
            "dx": dx, 
            "dy": dy, 
            "time": 0,  # Time for sine wave oscillation
            "oscillation_speed": oscillation_speed  # Speed of sine wave
        })
    glutTimerFunc(1000, spawn_snake, 0)



# Game update logic
def update(value):
    global ball_x, ball_y, ball_radius, health, score, upgrade, game_over

    if game_over or is_paused:
        glutTimerFunc(30, update, 0)
        return

    # Update snakes
    for snake in snakes[:]:
        # Increment time for sine wave (only used for curvy movement)
        snake["time"] += snake["oscillation_speed"]

        if score >= curvy_threshold:
            # Gradual amplitude: Increases every 5 points after curvy_threshold
            amplitude = min(((score - curvy_threshold) // 5) * 1.5, 25)  # Cap at 25
            if snake["dx"] != 0:  # Moving horizontally
                snake["x"] += snake["dx"]
                snake["y"] = max(0, min(window_height, snake["y"] + math.sin(snake["time"]) * amplitude))
            else:  # Moving vertically
                snake["y"] += snake["dy"]
                snake["x"] = max(0, min(window_width, snake["x"] + math.sin(snake["time"]) * amplitude))
        else:
            # Straight-line movement logic (original behavior)
            snake["x"] += snake["dx"]
            snake["y"] += snake["dy"]

        # Check if snake crosses the screen
        if snake["x"] < 0 or snake["x"] > window_width or snake["y"] < 0 or snake["y"] > window_height:
            snakes.remove(snake)
            score += 1
            if score >= 10:  # Upgrade threshold
                upgrade = True

        # Check collision with the ball
        if math.hypot(snake["x"] - ball_x, snake["y"] - ball_y) < ball_radius + snake["size"] // 2:
            health -= 10
            ball_radius += 2  # Ball grows larger
            snakes.remove(snake)
            if health <= 0:
                game_over = True

    # Update projectiles
    for projectile in projectiles[:]:
        projectile["x"] += projectile["dx"] * 5
        projectile["y"] += projectile["dy"] * 5

        # Remove projectiles that leave the screen
        if projectile["x"] < 0 or projectile["x"] > window_width or projectile["y"] < 0 or projectile["y"] > window_height:
            projectiles.remove(projectile)
            continue

        # Check collision with snakes
        for snake in snakes[:]:
            if math.hypot(projectile["x"] - snake["x"], projectile["y"] - snake["y"]) < snake["size"]:
                snakes.remove(snake)
                projectiles.remove(projectile)
                score += 1
                break

    glutPostRedisplay()
    glutTimerFunc(30, update, 0)



# Drawing the display
def display():
    glClear(GL_COLOR_BUFFER_BIT)
    if game_started:
        game_screen()
    else:
        starting_interface()

    # Draw the ball
    glColor3f(0.0, 1.0, 0.0)  # Green for ball
    draw_circle(ball_x, ball_y, ball_radius)

    # Draw snakes
    glColor3f(1.0, 0.0, 0.0)  # Red for snakes
    for snake in snakes:
        draw_snake(snake)

    # Draw projectiles
    glColor3f(1.0, 1.0, 1.0)  # White for projectiles
    for projectile in projectiles:
        draw_circle(projectile["x"], projectile["y"], 5)

    # Display health and score
    glColor3f(1.0, 1.0, 0.0)  # Yellow
    draw_text(10, window_height - 30, f"Score: {score}")
    draw_text(10, window_height - 50, f"Health: {health}")

    if game_over:
        draw_text(window_width // 2 - 80, window_height // 2, "GAME OVER", font=GLUT_BITMAP_TIMES_ROMAN_24)

    glutSwapBuffers()


def draw_text(x, y, text, font=GLUT_BITMAP_HELVETICA_18):
    glRasterPos2f(x, y)
    for char in text:
        glutBitmapCharacter(font, ord(char))

# Mouse input for throwing projectiles
def mouse_input(button, state, x, y):
    global projectiles

    global game_started ##


    if button == GLUT_LEFT_BUTTON and state == GLUT_DOWN and upgrade:
        # Calculate direction vector from ball to mouse click position
        mouse_x = x
        mouse_y = window_height - y  # Convert to OpenGL coordinates
        dx = mouse_x - ball_x
        dy = mouse_y - ball_y
        magnitude = math.sqrt(dx**2 + dy**2)
        if magnitude > 0:
            dx /= magnitude  # Normalize direction vector
            dy /= magnitude
        # Add projectile
        projectiles.append({"x": ball_x, "y": ball_y, "dx": dx, "dy": dy})
    
    if button == GLUT_LEFT_BUTTON and state == GLUT_DOWN: ##
        # Convert screen coordinates to normalized device coordinates
        ndc_x = (x / 500) * 2 - 1
        ndc_y = -((y / 500) * 2 - 1)

        # Check if Start button is clicked
        if -0.4 <= ndc_x <= 0.4 and 0.2 <= ndc_y <= 0.4:
            game_started = True
            glutPostRedisplay()

        # Check if Exit button is clicked
        if -0.4 <= ndc_x <= 0.4 and -0.4 <= ndc_y <= -0.2:
            sys.exit() ##

# Keyboard input
def keyboard_input(key, x, y):
    global ball_x, ball_y, is_paused
    if key == b'w':  # Move up
        ball_y = min(window_height, ball_y + ball_speed)
    elif key == b's':  # Move down
        ball_y = max(0, ball_y - ball_speed)
    elif key == b'a':  # Move left
        ball_x = max(0, ball_x - ball_speed)
    elif key == b'd':  # Move right
        ball_x = min(window_width, ball_x + ball_speed)
    elif key == b'p':  # Toggle pause
        is_paused = not is_paused
    glutPostRedisplay()

# Initialization
def init():
    glClearColor(0.0, 0.0, 0.0, 1.0)
    glPointSize(2)
    spawn_snake()
    glutTimerFunc(30, update, 0)

    glMatrixMode(GL_PROJECTION)##
    glLoadIdentity()
    gluOrtho2D(-1, 1, -1, 1)##

# Main program
glutInit()
glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)##
glutInitWindowSize(window_width, window_height)
glutCreateWindow(b"Serpent Strike")
glOrtho(0, window_width, 0, window_height, -1, 1)
init()
glutDisplayFunc(display)
glutMouseFunc(mouse_input)
glutKeyboardFunc(keyboard_input)
glutMainLoop()


IndentationError: expected an indented block after function definition on line 269 (360840680.py, line 270)

In [1]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import random
import math

# Window dimensions
window_width = 800
window_height = 600

# Game variables
ball_x = window_width // 2
ball_y = window_height // 2
ball_radius = 20
ball_speed = 5
health = 100
score = 0
snakes = []  # List of snakes
projectiles = []  # Projectiles shot by the ball
upgrade = False  # Ball upgrade status
game_over = False
is_paused = False

curvy_threshold = 5  # Snakes start moving curvy after reaching this score


# Drawing a pixel
def draw_pixel(x, y):
    glBegin(GL_POINTS)
    glVertex2f(x, y)
    glEnd()

# Midpoint Circle Algorithm
def draw_circle(xc, yc, r):
    x, y = 0, r
    d = 1 - r
    circle_points(xc, yc, x, y)
    while x < y:
        if d < 0:
            d += 2 * x + 3
        else:
            d += 2 * (x - y) + 5
            y -= 1
        x += 1
        circle_points(xc, yc, x, y)

def circle_points(xc, yc, x, y):
    points = [
        (xc + x, yc + y), (xc - x, yc + y),
        (xc + x, yc - y), (xc - x, yc - y),
        (xc + y, yc + x), (xc - y, yc + x),
        (xc + y, yc - x), (xc - y, yc - x)
    ]
    for point in points:
        draw_pixel(*point)

# Drawing a snake
def draw_snake(snake):
    # Define fixed dimensions for the snake
    body_length = 80  # Fixed length for the rectangle
    body_thickness = 20  # Fixed thickness for the rectangle

    # Determine the orientation based on movement direction
    if snake["dx"] != 0:  # Horizontal movement
        x1 = snake["x"] - body_length // 2
        x2 = snake["x"] + body_length // 2
        y1 = snake["y"] - body_thickness // 2
        y2 = snake["y"] + body_thickness // 2
    else:  # Vertical movement
        x1 = snake["x"] - body_thickness // 2
        x2 = snake["x"] + body_thickness // 2
        y1 = snake["y"] - body_length // 2
        y2 = snake["y"] + body_length // 2

    # Draw the rectangle body
    glBegin(GL_QUADS)
    glVertex2f(x1, y1)
    glVertex2f(x2, y1)
    glVertex2f(x2, y2)
    glVertex2f(x1, y2)
    glEnd()

    # Triangle head (snake's direction)
    if snake["dx"] > 0:  # Moving right
        triangle = [
            (x2, (y1 + y2) / 2),  # Left corner of the triangle base
            (x2 + body_thickness, y1),  # Bottom right of the base
            (x2 + body_thickness, y2)  # Top right of the base
        ]
    elif snake["dx"] < 0:  # Moving left
        triangle = [
            (x1, (y1 + y2) / 2),  # Right corner of the triangle base
            (x1 - body_thickness, y1),  # Bottom left of the base
            (x1 - body_thickness, y2)  # Top left of the base
        ]
    elif snake["dy"] > 0:  # Moving up
        triangle = [
            ((x1 + x2) / 2, y2),  # Bottom center of the triangle base
            (x1, y2 + body_thickness),  # Left corner of the triangle base
            (x2, y2 + body_thickness)  # Right corner of the triangle base
        ]
    else:  # Moving down
        triangle = [
            ((x1 + x2) / 2, y1),  # Top center of the triangle base
            (x1, y1 - body_thickness),  # Left corner of the triangle base
            (x2, y1 - body_thickness)  # Right corner of the triangle base
        ]

    # Draw the triangle head
    glBegin(GL_TRIANGLES)
    for vertex in triangle:
        glVertex2f(*vertex)
    glEnd()



# Spawn snakes
def spawn_snake(value=0):
    if not is_paused and not game_over:
        side = random.choice(['top', 'bottom', 'left', 'right'])
        size = random.randint(10, 30)
        speed = random.randint(2, 5)
        oscillation_speed = random.uniform(0.05, 0.2)  # Speed of the sine wave (only used after threshold)
        if side == 'top':
            x = random.randint(0, window_width)
            y = window_height
            dx, dy = 0, -speed
        elif side == 'bottom':
            x = random.randint(0, window_width)
            y = 0
            dx, dy = 0, speed
        elif side == 'left':
            x = 0
            y = random.randint(0, window_height)
            dx, dy = speed, 0
        elif side == 'right':
            x = window_width
            y = random.randint(0, window_height)
            dx, dy = -speed, 0
        snakes.append({
            "x": x, 
            "y": y, 
            "size": size, 
            "dx": dx, 
            "dy": dy, 
            "time": 0,  # Time for sine wave oscillation
            "oscillation_speed": oscillation_speed  # Speed of sine wave
        })
    glutTimerFunc(1000, spawn_snake, 0)



# Game update logic
def update(value):
    global ball_x, ball_y, ball_radius, health, score, upgrade, game_over

    if game_over or is_paused:
        glutTimerFunc(30, update, 0)
        return

    # Update snakes
    for snake in snakes[:]:
        # Increment time for sine wave (only used for curvy movement)
        snake["time"] += snake["oscillation_speed"]

        if score >= curvy_threshold:
            # Gradual amplitude: Increases every 5 points after curvy_threshold
            amplitude = min(((score - curvy_threshold) // 5) * 1.5, 25)  # Cap at 25
            if snake["dx"] != 0:  # Moving horizontally
                snake["x"] += snake["dx"]
                snake["y"] = max(0, min(window_height, snake["y"] + math.sin(snake["time"]) * amplitude))
            else:  # Moving vertically
                snake["y"] += snake["dy"]
                snake["x"] = max(0, min(window_width, snake["x"] + math.sin(snake["time"]) * amplitude))
        else:
            # Straight-line movement logic (original behavior)
            snake["x"] += snake["dx"]
            snake["y"] += snake["dy"]

        # Check if snake crosses the screen
        if snake["x"] < 0 or snake["x"] > window_width or snake["y"] < 0 or snake["y"] > window_height:
            snakes.remove(snake)
            score += 1
            if score >= 10:  # Upgrade threshold
                upgrade = True

        # Check collision with the ball
        if math.hypot(snake["x"] - ball_x, snake["y"] - ball_y) < ball_radius + snake["size"] // 2:
            health -= 10
            ball_radius += 2  # Ball grows larger
            snakes.remove(snake)
            if health <= 0:
                game_over = True

    # Update projectiles
    for projectile in projectiles[:]:
        projectile["x"] += projectile["dx"] * 5
        projectile["y"] += projectile["dy"] * 5

        # Remove projectiles that leave the screen
        if projectile["x"] < 0 or projectile["x"] > window_width or projectile["y"] < 0 or projectile["y"] > window_height:
            projectiles.remove(projectile)
            continue

        # Check collision with snakes
        for snake in snakes[:]:
            if math.hypot(projectile["x"] - snake["x"], projectile["y"] - snake["y"]) < snake["size"]:
                snakes.remove(snake)
                projectiles.remove(projectile)
                score += 1
                break

    glutPostRedisplay()
    glutTimerFunc(30, update, 0)



# Drawing the display
def display():
    glClear(GL_COLOR_BUFFER_BIT)

    # Draw the ball
    glColor3f(0.0, 1.0, 0.0)  # Green for ball
    draw_circle(ball_x, ball_y, ball_radius)

    # Draw snakes
    glColor3f(1.0, 0.0, 0.0)  # Red for snakes
    for snake in snakes:
        draw_snake(snake)

    # Draw projectiles
    glColor3f(1.0, 1.0, 1.0)  # White for projectiles
    for projectile in projectiles:
        draw_circle(projectile["x"], projectile["y"], 5)

    # Display health and score
    glColor3f(1.0, 1.0, 0.0)  # Yellow
    draw_text(10, window_height - 30, f"Score: {score}")
    draw_text(10, window_height - 50, f"Health: {health}")

    if game_over:
        draw_text(window_width // 2 - 80, window_height // 2, "GAME OVER", font=GLUT_BITMAP_TIMES_ROMAN_24)

    glutSwapBuffers()


def draw_text(x, y, text, font=GLUT_BITMAP_HELVETICA_18):
    glRasterPos2f(x, y)
    for char in text:
        glutBitmapCharacter(font, ord(char))

# Mouse input for throwing projectiles
def mouse_input(button, state, x, y):
    global projectiles
    if button == GLUT_LEFT_BUTTON and state == GLUT_DOWN and upgrade:
        # Calculate direction vector from ball to mouse click position
        mouse_x = x
        mouse_y = window_height - y  # Convert to OpenGL coordinates
        dx = mouse_x - ball_x
        dy = mouse_y - ball_y
        magnitude = math.sqrt(dx**2 + dy**2)
        if magnitude > 0:
            dx /= magnitude  # Normalize direction vector
            dy /= magnitude
        # Add projectile
        projectiles.append({"x": ball_x, "y": ball_y, "dx": dx, "dy": dy})


def reset_game():##
    global ball_x, ball_y, ball_radius, health, score, upgrade, game_over, snakes, projectiles
    ball_x, ball_y = window_width // 2, window_height // 2
    ball_radius = 20
    health = 100
    score = 0
    upgrade = False
    game_over = False
    snakes.clear()
    projectiles.clear()
    spawn_snake()##


# Keyboard input
def keyboard_input(key, x, y):
    global ball_x, ball_y, is_paused
    if key == b'w':  # Move up
        ball_y = min(window_height, ball_y + ball_speed)
    elif key == b's':  # Move down
        ball_y = max(0, ball_y - ball_speed)
    elif key == b'a':  # Move left
        ball_x = max(0, ball_x - ball_speed)
    elif key == b'd':  # Move right
        ball_x = min(window_width, ball_x + ball_speed)
    elif key == b'r' and game_over:  ##
        reset_game()##
    elif key == b'p':  # Toggle pause
        is_paused = not is_paused
    glutPostRedisplay()

# Initialization
def init():
    glClearColor(0.0, 0.0, 0.0, 1.0)
    glPointSize(2)
    spawn_snake()
    glutTimerFunc(30, update, 0)

# Main program
glutInit()
glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB)
glutInitWindowSize(window_width, window_height)
glutCreateWindow(b"Serpent Strike")
glOrtho(0, window_width, 0, window_height, -1, 1)
init()
glutDisplayFunc(display)
glutMouseFunc(mouse_input)
glutKeyboardFunc(keyboard_input)
glutMainLoop()


: 

In [ ]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import random
import math

# Window dimensions
window_width = 800
window_height = 600

# Game variables
ball_x = window_width // 2
ball_y = window_height // 2
ball_radius = 20
ball_speed = 5
health = 100
score = 0
snakes = []  # List of snakes
projectiles = []  # Projectiles shot by the ball
upgrade = False  # Ball upgrade status
game_over = False
is_paused = False
is_game_started = False  # Flag to check if the game has started

curvy_threshold = 5  # Snakes start moving curvy after reaching this score

# Drawing a pixel
def draw_pixel(x, y):
    glBegin(GL_POINTS)
    glVertex2f(x, y)
    glEnd()

# Midpoint Circle Algorithm
def draw_circle(xc, yc, r):
    x, y = 0, r
    d = 1 - r
    circle_points(xc, yc, x, y)
    while x < y:
        if d < 0:
            d += 2 * x + 3
        else:
            d += 2 * (x - y) + 5
            y -= 1
        x += 1
        circle_points(xc, yc, x, y)

def circle_points(xc, yc, x, y):
    points = [
        (xc + x, yc + y), (xc - x, yc + y),
        (xc + x, yc - y), (xc - x, yc - y),
        (xc + y, yc + x), (xc - y, yc + x),
        (xc + y, yc - x), (xc - y, yc - x)
    ]
    for point in points:
        draw_pixel(*point)

# Drawing a snake
def draw_snake(snake):
    body_length = 80
    body_thickness = 20
    if snake["dx"] != 0:
        x1 = snake["x"] - body_length // 2
        x2 = snake["x"] + body_length // 2
        y1 = snake["y"] - body_thickness // 2
        y2 = snake["y"] + body_thickness // 2
    else:
        x1 = snake["x"] - body_thickness // 2
        x2 = snake["x"] + body_thickness // 2
        y1 = snake["y"] - body_length // 2
        y2 = snake["y"] + body_length // 2

    glBegin(GL_QUADS)
    glVertex2f(x1, y1)
    glVertex2f(x2, y1)
    glVertex2f(x2, y2)
    glVertex2f(x1, y2)
    glEnd()

    if snake["dx"] > 0:
        triangle = [
            (x2, (y1 + y2) / 2), 
            (x2 + body_thickness, y1),  
            (x2 + body_thickness, y2)  
        ]
    elif snake["dx"] < 0:
        triangle = [
            (x1, (y1 + y2) / 2), 
            (x1 - body_thickness, y1),  
            (x1 - body_thickness, y2)  
        ]
    elif snake["dy"] > 0:
        triangle = [
            ((x1 + x2) / 2, y2),  
            (x1, y2 + body_thickness),  
            (x2, y2 + body_thickness)  
        ]
    else:
        triangle = [
            ((x1 + x2) / 2, y1),  
            (x1, y1 - body_thickness),  
            (x2, y1 - body_thickness)  
        ]

    glBegin(GL_TRIANGLES)
    for vertex in triangle:
        glVertex2f(*vertex)
    glEnd()

# Spawn snakes
def spawn_snake(value=0):
    if is_game_started and not is_paused and not game_over:
        side = random.choice(['top', 'bottom', 'left', 'right'])
        size = random.randint(10, 30)
        speed = random.randint(2, 5)
        oscillation_speed = random.uniform(0.05, 0.2)
        if side == 'top':
            x = random.randint(0, window_width)
            y = window_height
            dx, dy = 0, -speed
        elif side == 'bottom':
            x = random.randint(0, window_width)
            y = 0
            dx, dy = 0, speed
        elif side == 'left':
            x = 0
            y = random.randint(0, window_height)
            dx, dy = speed, 0
        elif side == 'right':
            x = window_width
            y = random.randint(0, window_height)
            dx, dy = -speed, 0
        snakes.append({
            "x": x, 
            "y": y, 
            "size": size, 
            "dx": dx, 
            "dy": dy, 
            "time": 0,  
            "oscillation_speed": oscillation_speed  
        })
    glutTimerFunc(1000, spawn_snake, 0)

# Draw the start screen
def draw_start_screen():
    glColor3f(1.0, 1.0, 1.0)  # White text
    draw_text(window_width // 2 - 100, window_height // 2 + 50, "Serpent Strike", font=GLUT_BITMAP_HELVETICA_18)
    
    # Start Button
    draw_rect(window_width // 2 - 60, window_height // 2, 120, 40, "Start")
    
    # Exit Button
    draw_rect(window_width // 2 - 60, window_height // 2 - 60, 120, 40, "Exit")

# Draw a rectangle for the button
def draw_rect(x, y, width, height, text):
    glColor3f(0.0, 0.0, 1.0)  # Blue for button
    glBegin(GL_QUADS)
    glVertex2f(x, y)
    glVertex2f(x + width, y)
    glVertex2f(x + width, y + height)
    glVertex2f(x, y + height)
    glEnd()
    
    draw_text(x + 10, y + height // 2 - 10, text, font=GLUT_BITMAP_HELVETICA_18)

# Mouse input for the start screen and game
def mouse_input(button, state, x, y):
    global is_game_started
    if button == GLUT_LEFT_BUTTON and state == GLUT_DOWN:
        if not is_game_started:
            # Check if clicked on "Start" button
            if window_width // 2 - 60 <= x <= window_width // 2 + 60 and window_height // 2 <= y <= window_height // 2 + 40:
                is_game_started = True
                spawn_snake()
            # Check if clicked on "Exit" button
            elif window_width // 2 - 60 <= x <= window_width // 2 + 60 and window_height // 2 - 60 <= y <= window_height // 2 - 20:
                glutLeaveMainLoop()

# Drawing the display
def display():
    glClear(GL_COLOR_BUFFER_BIT)

    if not is_game_started:
        draw_start_screen()
    else:
        # Draw the ball
        glColor3f(0.0, 1.0, 0.0)  # Green for ball
        draw_circle(ball_x, ball_y, ball_radius)

        # Draw snakes
        glColor3f(1.0, 0.0, 0.0)  # Red for snakes
        for snake in snakes:
            draw_snake(snake)

        # Draw projectiles
        glColor3f(1.0, 1.0, 1.0)  # White for projectiles
        for projectile in projectiles:
            draw_circle(projectile["x"], projectile["y"], 5)

        # Display health and score
        glColor3f(1.0, 1.0, 0.0)  # Yellow
        draw_text(10, window_height - 30, f"Score: {score}")
        draw_text(10, window_height - 50, f"Health: {health}")

        if game_over:
            draw_text(window_width // 2 - 80, window_height // 2, "GAME OVER", font=GLUT_BITMAP_TIMES_ROMAN_24)

    glutSwapBuffers()

def draw_text(x, y, text, font=GLUT_BITMAP_HELVETICA_18):
    glRasterPos2f(x, y)
    for char in text:
        glutBitmapCharacter(font, ord(char))

# Initialization
def init():
    glClearColor(0.0, 0.0, 0.0, 1.0)
    glPointSize(2)
    glutTimerFunc(30, update, 0)

# Main program
glutInit()
glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB)
glutInitWindowSize(window_width, window_height)
glutCreateWindow(b"Serpent Strike")
glOrtho(0, window_width, 0, window_height, -1, 1)
init()
glutDisplayFunc(display)
glutMouseFunc(mouse_input)
glutKeyboardFunc(keyboard_input)
glutMainLoop()
